## Seleção Ubots : Cientista de Dados Júnior

- Preencha os exercícios abaixo e depois compartilhe o seu documento no Google Colab com o email heukirne@gmail.com
- Não é obrigatório o preenchimento de todos os exercícios.

### Fazendo Download dos Arquivos

In [1]:
!wget -c http://gemeos.org/henrique/selecao_dados.zip -O selecao_dados.zip
!unzip -o selecao_dados.zip

--2021-01-08 20:42:47--  http://gemeos.org/henrique/selecao_dados.zip
Resolving gemeos.org (gemeos.org)... 187.45.195.133
Connecting to gemeos.org (gemeos.org)|187.45.195.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 562353 (549K) [application/zip]
Saving to: ‘selecao_dados.zip’

selecao_dados.zip   100%[===================>] 549.17K   622KB/s    in 0.9s    

2021-01-08 20:42:49 (622 KB/s) - ‘selecao_dados.zip’ saved [562353/562353]

Archive:  selecao_dados.zip
  inflating: regressao_1.csv         
  inflating: regressao_2.csv         
  inflating: classificacao_1.csv     
  inflating: classificacao_2.csv     
  inflating: agrupamento.csv         
  inflating: centroides_iniciais.csv  


In [28]:
#Libs gerais
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score


#Libs para regressão
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from math import sqrt
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RepeatedKFold


## Clusterização

### Melhor Agrupamento

Considerando os dados no arquivo agrupamento.csv, execute o algoritmo K-Means tendo como posição inicial dos centroides as K primeiras linhas do arquivo centroides.csv. Por exemplo, para executar o algoritmo com 3 clusters considere que os centroides iniciais são determinados pelas 3 primeiras linhas do arquivo centroides_iniciais.csv. Determine:



a) Qual a quantidade ideal de clusters?

### Função de Custo

Considerando os dados no arquivo agrupamento.csv, execute o algoritmo K-Means tendo como posição inicial dos centroides as K primeiras linhas do arquivo centroides.csv. Por exemplo, para executar o algoritmo com 3 clusters considere que os centroides iniciais são determinados pelas 3 primeiras linhas do arquivo centroides_iniciais.csv. Determine:



b) Qual o valor final da função de custo sendo minimizada pelo algoritmo K-Means para a quantidade ideal de clusters determinada na questão anterior (a)?

## Classificação

### Árvore de Decisão Partição de Treino

Considerando os dados do arquivo classificacao_1.csv, treine o modelo Árvore de Decisão (profundidade máxima de 5 níveis): 1) utilizando como metodologia de validação cruzada o k-Fold com k=10 (sequencial, isto é, utilizando a ordem dos dados para gerar as folds). Responda: 

a) Qual é a média da métrica F1 Score nas partições de treino?

### Árvore de Decisão Partições de Teste

Considerando os dados do arquivo classificacao_1.csv, treine o modelo Árvore de Decisão (profundidade máxima de 5 níveis) utilizando como metodologia de validação cruzada o k-Fold com k=10. Responda: 


b) Qual é a média da métrica F1 Score nas partições de validação?

### Naive Bayes Treino

Considerando os dados do arquivo classificacao_2.csv, treine o modelo Naive Bayes utilizando como metodologia de validação cruzada o k-Fold com k=5 (sequencial, isto é, utilizando a ordem dos dados para gerar as folds). Responda: 



- a) Qual é a média da métrica AUC nas partições de treino?
- b) Qual é a média da métrica AUC nas partições de validação?

## Regressão

Considerando os dados do arquivo regressao_1.csv, treine o modelo KNN (K=100, distância: euclidiana) utilizando como metodologia de validação cruzada o K-Fold com K=5 (sequencial, isto é, utilizando a ordem dos dados para gerar as folds). Responda:



- a) Qual é a média da métrica R² nas partições de treino?
- b) Qual é a média da métrica R² nas partições de validação?

In [3]:
#Captura o arquivo csv
df = pd.read_csv('regressao_1.csv')

#Apresenta os dados da tabela
df.head()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,v_15,v_16,target
0,0.41175,-0.88594,1.56763,-1.23920,-0.50609,0.40241,-0.19910,-0.36066,0.79811,-1.31550,-0.97004,0.25646,-0.97230,0.57444,0.86559,0.26008,59.426681
1,-0.18248,0.25817,-2.06830,0.30318,0.55240,1.14899,-1.06234,-1.05560,-0.39798,1.07560,0.32977,1.39860,0.13139,1.66229,-1.64970,-1.19903,-124.336003
2,0.16163,-0.19307,0.60995,0.75154,-0.34150,-1.07175,0.82420,-0.70458,0.15541,-0.41075,-0.36719,-0.40403,-1.36095,-0.32300,-1.80617,-2.36065,-135.453843
3,0.77408,-0.22030,-1.50507,0.26270,-0.43915,0.61604,-0.07312,1.05734,0.74734,-0.02001,-0.53163,-0.01862,-1.02064,0.20504,-0.46479,0.51637,15.301539
4,0.56369,-2.03858,1.00566,-0.22191,1.87744,0.92188,1.32863,1.23291,0.64139,-0.51216,-0.11285,1.36791,0.16337,0.17806,-0.79946,0.49963,0.344680


In [4]:
#Excluir valores duplicados
df = df.drop_duplicates(subset=["v_1", "v_2","v_3", "v_4", "v_5", "v_6", "v_7","v_8", "v_9", "v_10","v_11", "v_12","v_13", "v_14", "v_15", "v_16"])

#Dimensões do dataset
df.shape

(2500, 17)

In [5]:
#Verifica valores nulos
df.isnull().sum()

v_1       0
v_2       0
v_3       0
v_4       0
v_5       0
v_6       0
v_7       0
v_8       0
v_9       0
v_10      0
v_11      0
v_12      0
v_13      0
v_14      0
v_15      0
v_16      0
target    0
dtype: int64

In [7]:
#Separa 80% para treinamento e 20% para avaliação
train , test = train_test_split(df, test_size = 0.2)

#Separa atributos e rotulos do treinamento
x_train = train.drop('target', axis=1)
y_train = train['target']

#Separa atributos e rotulos do teste
x_test = train.drop('target', axis=1)
y_test = train['target']


In [8]:
#Normalização dos dados
scaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train_scaled)

x_test_scaled = scaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test_scaled)

In [15]:
#Definido parâmetros
K = 100      # Número de vizinhos
D = 2        # Manhattan_distance p = 1 and euclidean_distance p = 2)
CV = 5       # Quantidade de folds

#Instância o modelo
model = neighbors.KNeighborsRegressor(n_neighbors = K, p = D)

#Treino
model.fit(x_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=100, p=2,
                    weights='uniform')

#### **a) Qual é a média da métrica R² nas partições de treino?**

In [16]:
#Treina o modelo
cv_scores = cross_val_score(model, x_train, y_train, cv = CV, scoring='r2').mean()
cv_scores

0.6483841567304603

#### **b) Qual é a média da métrica R² nas partições de validação?**

In [17]:
cv_scores = cross_val_score(model, x_test, y_test, cv = CV, scoring='r2').mean()
cv_scores

0.6483841567304603

Considerando os dados do arquivo regressão_2.csv, treine o modelo Elastic Net (parâmetro de penalização: 0.01 (alpha no scikit-learn, lambda no glmnet), parâmetro de mistura: 0.5 (l1_ratio no scikit-learn, alpha no glmnet)) utilizando como metodologia de validação cruzada o K-Fold com K=10 (sequencial, isto é, utilizando a ordem dos dados para gerar as folds). Responda:



- a) Qual é a média da métrica MSE nas partições de treino?
- b) Qual é a média da métrica MSE nas partições de validação?

In [18]:
#Captura o arquivo csv
df = pd.read_csv('regressao_2.csv')

#Apresenta os dados da tabela
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,target
0,0.928565,-0.719700,-0.193906,-0.677486,-1.179784,-0.866473,-0.527861,-0.363196,-0.753120,-1.032732,-1.068407
1,-1.262432,-0.803072,-0.026298,0.070038,-1.223663,0.169027,0.677987,-0.170260,0.332584,0.504562,-0.476471
2,0.117189,0.853866,1.582002,1.481968,1.720371,0.209258,0.148371,1.694372,1.300706,1.236036,-6.258761
3,-0.148722,-0.133637,0.020990,-0.261777,-0.648657,-0.344005,0.689685,0.503188,0.034081,-0.003327,4.029771
4,-1.589064,-1.120601,-2.323954,-1.236945,-0.683011,-0.468862,-0.810920,-2.981145,-3.196629,-0.876743,-18.675821


In [20]:
#Excluir valores duplicados
df = df.drop_duplicates(subset=["V1", "V2","V3", "V4", "V5", "V6", "V7","V8", "V9", "V10"])

#Dimensões do dataset
df.shape

(1000, 11)

In [21]:
#Separa 80% para treinamento e 20% para avaliação
train , test = train_test_split(df, test_size = 0.2)

#Separa atributos e rotulos do treinamento
x_train = train.drop('target', axis=1)
y_train = train['target']

#Separa atributos e rotulos do teste
x_test = train.drop('target', axis=1)
y_test = train['target']

In [22]:
#Normalização dos dados
scaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train_scaled)

x_test_scaled = scaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test_scaled)

In [40]:
#Definido parâmetros

#Instância o modelo
model =  ElasticNet(alpha=0.01, l1_ratio=0.5)

#Treino
# define model evaluation method
cv = RepeatedKFold(n_splits=10)

# evaluate model
scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_absolute_error', cv=10)
# force scores to be positive

scores = np.absolute(scores)
print('Mean MSE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Mean MSE: 8.362 (0.658)


In [36]:
cv

RepeatedKFold(n_repeats=10, n_splits=10, random_state=None)

#### **a) Qual é a média da métrica MSE nas partições de treino?**


#### **b) Qual é a média da métrica MSE nas partições de validação?**

